In [113]:
import pandas as pd
import numpy as np
import re
import datetime
from pandas.core.frame import DataFrame
import matplotlib.pyplot as plt
import math
import statistics
from statistics import mean

In [114]:
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
plt.style.use('ggplot')
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline, make_pipeline
from scipy.stats import skew
from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import Imputer
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.isotonic import IsotonicRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.linear_model import ElasticNet, SGDRegressor, BayesianRidge
from sklearn.kernel_ridge import KernelRidge
from xgboost.sklearn import XGBRegressor
from scipy import stats

# Pre-processing

## Training set

In [5]:
filename='./Data/df_train1.csv'
df_train1 = pd.read_csv(filename)

In [6]:
df_train1

,playtime_forever,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews,pur_rel_itv
0,0.000000,False,3700,"Adventure,Casual,Indie","Single-player,Steam Trading Cards,Steam Cloud","Indie,Adventure,Story Rich,Casual,Atmospheric,...",2018-07-02,2013-12-10,372.0,96.0,1665
1,0.016667,True,0,RPG,"Single-player,Partial Controller Support","Mod,Utilities,RPG,Game Development,Singleplaye...",2016-11-26,2015-08-12,23.0,0.0,472
2,0.000000,False,5000,"Adventure,Casual,Indie","Single-player,Full controller support,Steam Tr...","Point & Click,Adventure,Story Rich,Comedy,Indi...",2018-07-02,2014-01-28,3018.0,663.0,1616
3,1.533333,False,9900,"Action,RPG","Single-player,Multi-player,Steam Achievements,...","Medieval,RPG,Open World,Strategy,Sandbox,Actio...",2016-11-28,2010-03-31,63078.0,1746.0,2434
4,22.333333,False,4800,"Action,Indie,Strategy","Single-player,Co-op,Steam Achievements,Full co...","Tower Defense,Co-op,Action,Strategy,Online Co-...",2018-03-04,2012-07-30,8841.0,523.0,2043
...,...,...,...,...,...,...,...,...,...,...,...
335,0.783333,False,8800,"Action,Simulation","Single-player,Multi-player,Online Multi-Player...","Simulation,Action,VR,Flight",2017-11-24,2016-12-20,150.0,91.0,339
336,0.000000,False,6800,"Action,Adventure","Single-player,Steam Achievements,Full controll...","Batman,Action,Open World,Superhero,Stealth,Adv...",2018-08-15,2015-06-23,19008.0,4849.0,1149
337,0.000000,False,8300,"Action,Adventure,RPG","Single-player,Steam Achievements,Full controll...","Action,Hack and Slash,Adventure,RPG,Open World...",2018-01-30,2015-11-05,5099.0,1719.0,817
338,0.000000,False,6800,"Action,Adventure","Single-player,Steam Achievements,Full controll...","Action,FPS,Adventure,Shooter,Cyberpunk,Robots,...",2017-09-23,2016-06-03,718.0,159.0,477


### Read One-hot matrix from csv - process see 1118_genres&categories&tags.ipynb

In [7]:
filename='./Data/train_One_hot_genres.csv'
dfg_train = pd.read_csv(filename)

In [8]:
df_train2 = pd.concat([df_train1, dfg_train], axis=1, sort=False)
df_train2

,playtime_forever,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews,...,Modeling,Nudity,RPG,Racing,SexualContent,Simulation,Sports,Strategy,Utilities,Violent
0,0.000000,False,3700,"Adventure,Casual,Indie","Single-player,Steam Trading Cards,Steam Cloud","Indie,Adventure,Story Rich,Casual,Atmospheric,...",2018-07-02,2013-12-10,372.0,96.0,...,0,0,0,0,0,0,0,0,0,0
1,0.016667,True,0,RPG,"Single-player,Partial Controller Support","Mod,Utilities,RPG,Game Development,Singleplaye...",2016-11-26,2015-08-12,23.0,0.0,...,0,0,1,0,0,0,0,0,0,0
2,0.000000,False,5000,"Adventure,Casual,Indie","Single-player,Full controller support,Steam Tr...","Point & Click,Adventure,Story Rich,Comedy,Indi...",2018-07-02,2014-01-28,3018.0,663.0,...,0,0,0,0,0,0,0,0,0,0
3,1.533333,False,9900,"Action,RPG","Single-player,Multi-player,Steam Achievements,...","Medieval,RPG,Open World,Strategy,Sandbox,Actio...",2016-11-28,2010-03-31,63078.0,1746.0,...,0,0,1,0,0,0,0,0,0,0
4,22.333333,False,4800,"Action,Indie,Strategy","Single-player,Co-op,Steam Achievements,Full co...","Tower Defense,Co-op,Action,Strategy,Online Co-...",2018-03-04,2012-07-30,8841.0,523.0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,0.783333,False,8800,"Action,Simulation","Single-player,Multi-player,Online Multi-Player...","Simulation,Action,VR,Flight",2017-11-24,2016-12-20,150.0,91.0,...,0,0,0,0,0,1,0,0,0,0
336,0.000000,False,6800,"Action,Adventure","Single-player,Steam Achievements,Full controll...","Batman,Action,Open World,Superhero,Stealth,Adv...",2018-08-15,2015-06-23,19008.0,4849.0,...,0,0,0,0,0,0,0,0,0,0
337,0.000000,False,8300,"Action,Adventure,RPG","Single-player,Steam Achievements,Full controll...","Action,Hack and Slash,Adventure,RPG,Open World...",2018-01-30,2015-11-05,5099.0,1719.0,...,0,0,1,0,0,0,0,0,0,0
338,0.000000,False,6800,"Action,Adventure","Single-player,Steam Achievements,Full controll...","Action,FPS,Adventure,Shooter,Cyberpunk,Robots,...",2017-09-23,2016-06-03,718.0,159.0,...,0,0,0,0,0,0,0,0,0,0


## Testing set 

In [115]:
filename='./Data/test.csv'
df_test = pd.read_csv(filename)
df_test

,id,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews
0,0,False,3500,"Action,Adventure","Single-player,Full controller support","Action,Adventure,Horror,Third Person,Singlepla...","Oct 28, 2018",22-May-12,2607.0,1122.0
1,1,False,11600,"Action,Adventure,Strategy","Single-player,Multi-player,Online Multi-Player...","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Jul 20, 2019",24-Apr-18,5762.0,2235.0
2,2,False,2100,"Indie,Simulation,Strategy","Single-player,Steam Achievements,Steam Trading...","Strategy,Simulation,Indie,Political,Cold War,P...","Jul 16, 2019",20-Mar-17,687.0,133.0
3,3,False,3600,"Action,Strategy","Single-player,Multi-player,Co-op","Strategy,Action,Military,Tactical","Jan 30, 2018",17-Jul-07,67.0,39.0
4,4,False,3400,"Action,Adventure","Single-player,Co-op,Steam Achievements,Full co...","Open World,Action,Comedy,Co-op,Third-Person Sh...","Feb 24, 2017",22-Aug-13,40344.0,3708.0
...,...,...,...,...,...,...,...,...,...,...
85,85,False,5000,Action,"Single-player,Steam Achievements,Full controll...","Western,FPS,Action,Story Rich,Bullet Time,Shoo...","Mar 23, 2018",22-May-13,9334.0,516.0
86,86,False,3600,"Action,Adventure",Single-player,"Adventure,Action,Ninja,Stealth,Cute,Singleplay...","Mar 3, 2018",14-Sep-09,1245.0,176.0
87,87,False,11200,"Simulation,Strategy","Single-player,Multi-player,Cross-Platform Mult...","Grand Strategy,Strategy,Historical,Simulation,...","Feb 18, 2018",13-Aug-13,33128.0,6350.0
88,88,True,0,"Action,Adventure,RPG,Simulation,Sports,Strategy","Single-player,Multi-player,Co-op,Online Co-op,...","VR,RPG,Action,Sports,Simulation,Adventure,Stra...","Nov 19, 2017",19-Jun-17,5.0,0.0


# Use Mode to Fill Empty

In [116]:
df_train2['total_positive_reviews'].mode()
df_train2['total_negative_reviews'].mode()

0    0.0
dtype: float64

In [117]:
# handling empty value in total_positive_reviews and total_negative_reviews
df_test = df_test.fillna({'total_positive_reviews': df_test['total_positive_reviews'].mode(),'total_negative_reviews': df_test['total_negative_reviews'].mode()})

In [118]:
# handling empty value in purchase_date or release_date
df_test = df_test.fillna(method='ffill')

In [119]:
# date transform in release_date
months=[['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'],
        ['01' ,'02' ,'03' ,'04' ,'05' ,'06' ,'07' ,'08' ,'09' ,'10' ,'11' ,'12'] ]

index_d=0;
for d in df_test.release_date:
    # handling singe number day
    if len(d) == 8:
        d = '0'+ d
    
    # converting month into digital
    month=''.join(re.findall(r'[A-Za-z]', d))
#    print (month)
    for m in range(len(months[0][:])):
        if month == months[0][m]:
            month = months[1][m]
            d = d.replace(d[3:6], month)
            break
#    print (d)
    df_test.release_date.iloc[index_d] = d
    index_d = index_d + 1

In [120]:
# transfer release_date to date-type
date_format = '%d-%m-%y'
df_test['release_date'] = pd.to_datetime(df_test['release_date'],format = date_format)
df_test.release_date[0:6]

0   2012-05-22
1   2018-04-24
2   2017-03-20
3   2007-07-17
4   2013-08-22
5   2018-02-28
Name: release_date, dtype: datetime64[ns]

In [121]:
# date transform in purchase_date
index_d=0;

for d in df_test.purchase_date:
    d = d.replace(' ', '-')
    d = d.replace(',', '')
    
    # converting month into digital
    month=''.join(re.findall(r'[A-Za-z]', d))
    for m in range(len(months[0][:])):
        if month == months[0][m]:
            month = months[1][m]
            d = d.replace(d[0:3], month)
            break
            
    df_test.purchase_date.iloc[index_d] = d
    index_d = index_d + 1
#    print (d)

In [122]:
# transfer purchase_date to date-type
date_format_r = '%m-%d-%Y'
df_test['purchase_date'] = pd.to_datetime(df_test['purchase_date'],format = date_format_r)
df_test.purchase_date

0    2018-10-28
1    2019-07-20
2    2019-07-16
3    2018-01-30
4    2017-02-24
        ...    
85   2018-03-23
86   2018-03-03
87   2018-02-18
88   2017-11-19
89   2018-11-25
Name: purchase_date, Length: 90, dtype: datetime64[ns]

### compute pri

In [123]:
# compute the interval of purchase_date and release_date

df_test['pur_rel_itv'] = df_test['purchase_date']-df_test['release_date']

index_pri = 0
for p in df_test.pur_rel_itv:
#    print (p)
    df_test.pur_rel_itv[index_pri] = p.days
    index_pri = index_pri + 1


In [124]:
# if pri < 0, replace with mean
index_p = 0
for pri in df_test.pur_rel_itv:
    if pri < 0:
        df_test.pur_rel_itv.iloc[index_p] = df_test['pur_rel_itv'].mean()
    index_p = index_p + 1

In [125]:
df_test

,id,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews,pur_rel_itv
0,0,False,3500,"Action,Adventure","Single-player,Full controller support","Action,Adventure,Horror,Third Person,Singlepla...",2018-10-28,2012-05-22,2607.0,1122.0,2350
1,1,False,11600,"Action,Adventure,Strategy","Single-player,Multi-player,Online Multi-Player...","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...",2019-07-20,2018-04-24,5762.0,2235.0,452
2,2,False,2100,"Indie,Simulation,Strategy","Single-player,Steam Achievements,Steam Trading...","Strategy,Simulation,Indie,Political,Cold War,P...",2019-07-16,2017-03-20,687.0,133.0,848
3,3,False,3600,"Action,Strategy","Single-player,Multi-player,Co-op","Strategy,Action,Military,Tactical",2018-01-30,2007-07-17,67.0,39.0,3850
4,4,False,3400,"Action,Adventure","Single-player,Co-op,Steam Achievements,Full co...","Open World,Action,Comedy,Co-op,Third-Person Sh...",2017-02-24,2013-08-22,40344.0,3708.0,1282
...,...,...,...,...,...,...,...,...,...,...,...
85,85,False,5000,Action,"Single-player,Steam Achievements,Full controll...","Western,FPS,Action,Story Rich,Bullet Time,Shoo...",2018-03-23,2013-05-22,9334.0,516.0,1766
86,86,False,3600,"Action,Adventure",Single-player,"Adventure,Action,Ninja,Stealth,Cute,Singleplay...",2018-03-03,2009-09-14,1245.0,176.0,3092
87,87,False,11200,"Simulation,Strategy","Single-player,Multi-player,Cross-Platform Mult...","Grand Strategy,Strategy,Historical,Simulation,...",2018-02-18,2013-08-13,33128.0,6350.0,1650
88,88,True,0,"Action,Adventure,RPG,Simulation,Sports,Strategy","Single-player,Multi-player,Co-op,Online Co-op,...","VR,RPG,Action,Sports,Simulation,Adventure,Stra...",2017-11-19,2017-06-19,5.0,0.0,153


In [126]:
df_test.to_csv("./Data/df_test1_mode.csv",sep = ',',index = False)

### Read One-hot Matrix from csv - process see 1118_genres&categories&tags.ipynb

In [26]:
filename='./Data/test_One_hot_genres.csv'
dfg_test = pd.read_csv(filename)

In [27]:
df_test2 = pd.concat([df_test, dfg_test], axis=1, sort=False)
df_test2

,id,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews,...,Modeling,Nudity,RPG,Racing,SexualContent,Simulation,Sports,Strategy,Utilities,Violent
0,0,False,3500,"Action,Adventure","Single-player,Full controller support","Action,Adventure,Horror,Third Person,Singlepla...",2018-10-28,2012-05-22,2607.0,1122.0,...,0,0,0,0,0,0,0,0,0,0
1,1,False,11600,"Action,Adventure,Strategy","Single-player,Multi-player,Online Multi-Player...","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...",2019-07-20,2018-04-24,5762.0,2235.0,...,0,0,0,0,0,0,0,1,0,0
2,2,False,2100,"Indie,Simulation,Strategy","Single-player,Steam Achievements,Steam Trading...","Strategy,Simulation,Indie,Political,Cold War,P...",2019-07-16,2017-03-20,687.0,133.0,...,0,0,0,0,0,1,0,1,0,0
3,3,False,3600,"Action,Strategy","Single-player,Multi-player,Co-op","Strategy,Action,Military,Tactical",2018-01-30,2007-07-17,67.0,39.0,...,0,0,0,0,0,0,0,1,0,0
4,4,False,3400,"Action,Adventure","Single-player,Co-op,Steam Achievements,Full co...","Open World,Action,Comedy,Co-op,Third-Person Sh...",2017-02-24,2013-08-22,40344.0,3708.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,85,False,5000,Action,"Single-player,Steam Achievements,Full controll...","Western,FPS,Action,Story Rich,Bullet Time,Shoo...",2018-03-23,2013-05-22,9334.0,516.0,...,0,0,0,0,0,0,0,0,0,0
86,86,False,3600,"Action,Adventure",Single-player,"Adventure,Action,Ninja,Stealth,Cute,Singleplay...",2018-03-03,2009-09-14,1245.0,176.0,...,0,0,0,0,0,0,0,0,0,0
87,87,False,11200,"Simulation,Strategy","Single-player,Multi-player,Cross-Platform Mult...","Grand Strategy,Strategy,Historical,Simulation,...",2018-02-18,2013-08-13,33128.0,6350.0,...,0,0,0,0,0,1,0,1,0,0
88,88,True,0,"Action,Adventure,RPG,Simulation,Sports,Strategy","Single-player,Multi-player,Co-op,Online Co-op,...","VR,RPG,Action,Sports,Simulation,Adventure,Stra...",2017-11-19,2017-06-19,5.0,0.0,...,0,0,1,0,0,1,1,1,0,0


# Feature selection

In [28]:
for d in zip(df_train2, df_test2):
    if d[0] != d[1]:
        print (d)

('playtime_forever', 'id')


In [31]:
scaler = StandardScaler()

X = df_train2.drop(columns=['playtime_forever', 'is_free', 'genres', 'categories', 'tags', 'purchase_date','release_date'])
test_X = df_test2.drop(columns=['id', 'is_free', 'genres', 'categories', 'tags', 'purchase_date','release_date'])
y= df_train2.playtime_forever

X_scaled = scaler.fit(X).transform(X)
y_log = np.log(df_train2.playtime_forever + 1)
test_X_scaled = scaler.transform(test_X)

In [32]:
X_scaled

array([[-0.07482934, -0.35131016, -0.21882637, ..., -0.54562801,
        -0.05431254, -0.13403012],
       [-0.07874784, -0.36050227, -0.23019318, ..., -0.54562801,
        -0.05431254, -0.13403012],
       [-0.07345257, -0.28161873, -0.15169116, ..., -0.54562801,
        -0.05431254, -0.13403012],
       ...,
       [-0.06995769, -0.2268085 , -0.02665628, ..., -0.54562801,
        -0.05431254, -0.13403012],
       [-0.07154627, -0.34219707, -0.21136691, ..., -0.54562801,
        -0.05431254, -0.13403012],
       [-0.0681573 , -0.3370084 , -0.21811595, ...,  1.83275049,
        -0.05431254, -0.13403012]])

# Modeling & Evaluation 

In [33]:
# define cross validation stragegy RMSE
def rmse_cv(model,X,y):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=5))
    return rmse

In [34]:
models = [LinearRegression(),Ridge(),Lasso(),RandomForestRegressor(),GradientBoostingRegressor(),SVR(),LinearSVR(),
          ElasticNet(),SGDRegressor(),BayesianRidge(),KernelRidge(),
          ExtraTreesRegressor(),XGBRegressor(verbosity=0),AdaBoostRegressor()]

In [35]:
names = ["LR", "Ridge", "Lasso", "RF", "GBR", "SVR", "LinSVR", "Ela","SGD","Bay","Ker","Extra","Xgb","Ada"]
for name, model in zip(names, models):
    score = rmse_cv(model, X_scaled, y_log)
    print("{}: {:.6f}, {:.4f}".format(name,score.mean(),score.std()))

LR: 13355495097098.308594, 26710990194194.8203
Ridge: 0.865655, 0.1264
Lasso: 0.883973, 0.1248
RF: 0.813695, 0.1417
GBR: 0.874993, 0.1224
SVR: 0.844382, 0.1326
LinSVR: 0.968785, 0.1478
Ela: 0.883973, 0.1248
SGD: 1.082491, 0.4082
Bay: 0.825339, 0.1228
Ker: 6.026479, 4.5220
Extra: 0.844669, 0.1277
Xgb: 0.836758, 0.1187
Ada: 0.856076, 0.1006


## Fine Tuning Parameters

In [36]:
class grid():
    def __init__(self,model):
        self.model = model
    
    def grid_get(self,X,y,param_grid):
        grid_search = GridSearchCV(self.model,param_grid,cv=5, scoring="neg_mean_squared_error")
        grid_search.fit(X,y)
        print(grid_search.best_params_, np.sqrt(-grid_search.best_score_))
        grid_search.cv_results_['mean_test_score'] = np.sqrt(-grid_search.cv_results_['mean_test_score'])
        print(pd.DataFrame(grid_search.cv_results_)[['params','mean_test_score','std_test_score']])

In [70]:
# Ridge

grid(Ridge()).grid_get(X_scaled,y_log,{'alpha':[155,156,157,158,159,160,161,162,163,164,165],'solver':["auto", "svd", "cholesky", "lsqr", "sparse_cg"]})

{'alpha': 160, 'solver': 'sparse_cg'} 0.8295796987290535
                                   params  mean_test_score  std_test_score
0        {'alpha': 155, 'solver': 'auto'}         0.829607        0.211478
1         {'alpha': 155, 'solver': 'svd'}         0.829607        0.211478
2    {'alpha': 155, 'solver': 'cholesky'}         0.829607        0.211478
3        {'alpha': 155, 'solver': 'lsqr'}         0.829609        0.211436
4   {'alpha': 155, 'solver': 'sparse_cg'}         0.829591        0.211456
5        {'alpha': 156, 'solver': 'auto'}         0.829603        0.211528
6         {'alpha': 156, 'solver': 'svd'}         0.829603        0.211528
7    {'alpha': 156, 'solver': 'cholesky'}         0.829603        0.211528
8        {'alpha': 156, 'solver': 'lsqr'}         0.829606        0.211487
9   {'alpha': 156, 'solver': 'sparse_cg'}         0.829587        0.211506
10       {'alpha': 157, 'solver': 'auto'}         0.829600        0.211578
11        {'alpha': 157, 'solver': 'svd'}  

In [37]:
# RF - USABLE

grid(RandomForestRegressor()).grid_get(X_scaled,y_log,{'max_features': ["auto","sqrt","log2"]})

{'max_features': 'sqrt'} 0.8125165095569998
                     params  mean_test_score  std_test_score
0  {'max_features': 'auto'}         0.832448        0.281391
1  {'max_features': 'sqrt'}         0.812517        0.209459
2  {'max_features': 'log2'}         0.838938        0.229987


In [61]:
# Lasso

grid(Lasso()).grid_get(X_scaled,y_log,{'alpha': [0.041,0.042,0.043,0.044,0.045,0.046,0.047,0.048,0.049,0.05],'max_iter':[1000,10000]})

{'alpha': 0.044, 'max_iter': 1000} 0.8262875216183371
                                 params  mean_test_score  std_test_score
0    {'alpha': 0.041, 'max_iter': 1000}         0.826395        0.185235
1   {'alpha': 0.041, 'max_iter': 10000}         0.826395        0.185235
2    {'alpha': 0.042, 'max_iter': 1000}         0.826370        0.185298
3   {'alpha': 0.042, 'max_iter': 10000}         0.826370        0.185298
4    {'alpha': 0.043, 'max_iter': 1000}         0.826323        0.185262
5   {'alpha': 0.043, 'max_iter': 10000}         0.826323        0.185262
6    {'alpha': 0.044, 'max_iter': 1000}         0.826288        0.185235
7   {'alpha': 0.044, 'max_iter': 10000}         0.826288        0.185235
8    {'alpha': 0.045, 'max_iter': 1000}         0.826307        0.185297
9   {'alpha': 0.045, 'max_iter': 10000}         0.826307        0.185297
10   {'alpha': 0.046, 'max_iter': 1000}         0.826331        0.185362
11  {'alpha': 0.046, 'max_iter': 10000}         0.826331        0.1853

In [57]:
# GBR

grid(GradientBoostingRegressor()).grid_get(X_scaled,y_log,{'learning_rate':[0.006,0.007,0.008,0.009,0.01,0.011,0.012,0.013,0.014,0.015]})

{'learning_rate': 0.014} 0.8099964417320196
                     params  mean_test_score  std_test_score
0  {'learning_rate': 0.006}         0.826620        0.213151
1  {'learning_rate': 0.007}         0.821504        0.210832
2  {'learning_rate': 0.008}         0.817261        0.207691
3  {'learning_rate': 0.009}         0.814996        0.207440
4   {'learning_rate': 0.01}         0.812839        0.206206
5  {'learning_rate': 0.011}         0.811885        0.203259
6  {'learning_rate': 0.012}         0.811832        0.201584
7  {'learning_rate': 0.013}         0.811280        0.203535
8  {'learning_rate': 0.014}         0.809996        0.202545
9  {'learning_rate': 0.015}         0.811313        0.201181


In [40]:
# SVR - USABLE

grid(SVR()).grid_get(X_scaled,y_log,{'C':[2.5,3,3.5],'kernel':["linear","poly","rbf"],'degree':[2,3]})

{'C': 2.5, 'degree': 2, 'kernel': 'rbf'} 0.8449253700650498
                                         params  mean_test_score  \
0   {'C': 2.5, 'degree': 2, 'kernel': 'linear'}         0.942174   
1     {'C': 2.5, 'degree': 2, 'kernel': 'poly'}         0.989409   
2      {'C': 2.5, 'degree': 2, 'kernel': 'rbf'}         0.844925   
3   {'C': 2.5, 'degree': 3, 'kernel': 'linear'}         0.942174   
4     {'C': 2.5, 'degree': 3, 'kernel': 'poly'}         1.001107   
5      {'C': 2.5, 'degree': 3, 'kernel': 'rbf'}         0.844925   
6     {'C': 3, 'degree': 2, 'kernel': 'linear'}         0.942160   
7       {'C': 3, 'degree': 2, 'kernel': 'poly'}         0.994583   
8        {'C': 3, 'degree': 2, 'kernel': 'rbf'}         0.845127   
9     {'C': 3, 'degree': 3, 'kernel': 'linear'}         0.942160   
10      {'C': 3, 'degree': 3, 'kernel': 'poly'}         1.020591   
11       {'C': 3, 'degree': 3, 'kernel': 'rbf'}         0.845127   
12  {'C': 3.5, 'degree': 2, 'kernel': 'linear'}         

In [54]:
# Ela

grid(ElasticNet()).grid_get(X_scaled,y_log,{'alpha':[0.07,0.071,0.072,0.073,0.074,0.075,0.076,0.077,0.078,0.079]})

{'alpha': 0.074} 0.8265009358419071
             params  mean_test_score  std_test_score
0   {'alpha': 0.07}         0.826531        0.186284
1  {'alpha': 0.071}         0.826516        0.186477
2  {'alpha': 0.072}         0.826506        0.186669
3  {'alpha': 0.073}         0.826501        0.186858
4  {'alpha': 0.074}         0.826501        0.187046
5  {'alpha': 0.075}         0.826505        0.187233
6  {'alpha': 0.076}         0.826514        0.187417
7  {'alpha': 0.077}         0.826528        0.187599
8  {'alpha': 0.078}         0.826544        0.187783
9  {'alpha': 0.079}         0.826566        0.187963


In [44]:
# Bay - USABLE

grid(BayesianRidge()).grid_get(X_scaled,y_log,{'alpha_1':[0.0001,0.0005,0.001,0.01,0.05,0.1,0.5,1]})

{'alpha_1': 0.0001} 0.8344257887029795
                params  mean_test_score  std_test_score
0  {'alpha_1': 0.0001}         0.834426        0.208577
1  {'alpha_1': 0.0005}         0.834426        0.208577
2   {'alpha_1': 0.001}         0.834426        0.208577
3    {'alpha_1': 0.01}         0.834427        0.208576
4    {'alpha_1': 0.05}         0.834430        0.208574
5     {'alpha_1': 0.1}         0.834435        0.208571
6     {'alpha_1': 0.5}         0.834472        0.208547
7       {'alpha_1': 1}         0.834517        0.208518


In [45]:
# Extra - USABLE

grid(ExtraTreesRegressor()).grid_get(X_scaled,y_log,{'max_features':["auto","sqrt","log2"],'bootstrap':[True]})

{'bootstrap': True, 'max_features': 'sqrt'} 0.8387641582661924
                                        params  mean_test_score  \
0  {'bootstrap': True, 'max_features': 'auto'}         0.869749   
1  {'bootstrap': True, 'max_features': 'sqrt'}         0.838764   
2  {'bootstrap': True, 'max_features': 'log2'}         0.850643   

   std_test_score  
0        0.252047  
1        0.190807  
2        0.284611  


In [54]:
# Ker

grid(KernelRidge()).grid_get(X_scaled,y_log,{'alpha':[0.1,0.2,0.3,0.4], 'kernel':["linear","polynomial"], 'degree':[2,3,4],'coef0':[0.1,0.5,0.8,0.9]})

{'alpha': 0.1, 'coef0': 0.9, 'degree': 2, 'kernel': 'polynomial'} 0.8341496409589338
                                               params  mean_test_score  \
0   {'alpha': 0.1, 'coef0': 0.1, 'degree': 2, 'ker...         0.869065   
1   {'alpha': 0.1, 'coef0': 0.1, 'degree': 2, 'ker...         0.853602   
2   {'alpha': 0.1, 'coef0': 0.1, 'degree': 3, 'ker...         0.869065   
3   {'alpha': 0.1, 'coef0': 0.1, 'degree': 3, 'ker...         0.874624   
4   {'alpha': 0.1, 'coef0': 0.1, 'degree': 4, 'ker...         0.869065   
..                                                ...              ...   
91  {'alpha': 0.4, 'coef0': 0.9, 'degree': 2, 'ker...         0.843385   
92  {'alpha': 0.4, 'coef0': 0.9, 'degree': 3, 'ker...         0.849509   
93  {'alpha': 0.4, 'coef0': 0.9, 'degree': 3, 'ker...         0.839915   
94  {'alpha': 0.4, 'coef0': 0.9, 'degree': 4, 'ker...         0.849509   
95  {'alpha': 0.4, 'coef0': 0.9, 'degree': 4, 'ker...         0.839642   

    std_test_score  
0    

In [46]:
# Xgb - USABLE 

grid(XGBRegressor()).grid_get(X_scaled,y_log,{'booster':["gbtree", "dart"],'tree_method':["auto", "exact", "approx", "hist"],'verbosity':[0]})

{'booster': 'dart', 'tree_method': 'auto', 'verbosity': 0} 0.8451390391112796
                                              params  mean_test_score  \
0  {'booster': 'gbtree', 'tree_method': 'auto', '...         0.845139   
1  {'booster': 'gbtree', 'tree_method': 'exact', ...         0.845139   
2  {'booster': 'gbtree', 'tree_method': 'approx',...         0.869342   
3  {'booster': 'gbtree', 'tree_method': 'hist', '...         0.851393   
4  {'booster': 'dart', 'tree_method': 'auto', 've...         0.845139   
5  {'booster': 'dart', 'tree_method': 'exact', 'v...         0.845139   
6  {'booster': 'dart', 'tree_method': 'approx', '...         0.869342   
7  {'booster': 'dart', 'tree_method': 'hist', 've...         0.851393   

   std_test_score  
0        0.205273  
1        0.205273  
2        0.200882  
3        0.188872  
4        0.205273  
5        0.205273  
6        0.200882  
7        0.188871  


In [50]:
# Ada - USABLE

grid(AdaBoostRegressor()).grid_get(X_scaled,y_log,{'learning_rate':[0.016,0.017,0.018,0.019,0.02,0.021,0.022,0.023,0.024],'loss':["linear", "square", "exponential"]})

{'learning_rate': 0.018, 'loss': 'exponential'} 0.7853757206465998
                                             params  mean_test_score  \
0        {'learning_rate': 0.016, 'loss': 'linear'}         0.799087   
1        {'learning_rate': 0.016, 'loss': 'square'}         0.811639   
2   {'learning_rate': 0.016, 'loss': 'exponential'}         0.796562   
3        {'learning_rate': 0.017, 'loss': 'linear'}         0.800070   
4        {'learning_rate': 0.017, 'loss': 'square'}         0.796481   
5   {'learning_rate': 0.017, 'loss': 'exponential'}         0.804673   
6        {'learning_rate': 0.018, 'loss': 'linear'}         0.791055   
7        {'learning_rate': 0.018, 'loss': 'square'}         0.806322   
8   {'learning_rate': 0.018, 'loss': 'exponential'}         0.785376   
9        {'learning_rate': 0.019, 'loss': 'linear'}         0.812811   
10       {'learning_rate': 0.019, 'loss': 'square'}         0.805525   
11  {'learning_rate': 0.019, 'loss': 'exponential'}         0.804853 

# Ensemble Methods

In [71]:
class AverageWeight(BaseEstimator, RegressorMixin):
    def __init__(self,mod,weight):
        self.mod = mod
        self.weight = weight
        
    def fit(self,X,y):
        self.models_ = [clone(x) for x in self.mod]
        for model in self.models_:
            model.fit(X,y)
        return self
    
    def predict(self,X):
        w = list()
        pred = np.array([model.predict(X) for model in self.models_])
        # for every data point, single model prediction times weight, then add them together
        for data in range(pred.shape[1]):
            single = [pred[model,data]*weight for model,weight in zip(range(pred.shape[0]),self.weight)]
            w.append(np.sum(single))
        return w

In [93]:
# sorted by performance(good -> bad)
ada = AdaBoostRegressor(learning_rate = 0.06, loss = 'square')
rf = RandomForestRegressor(max_features = 'log2')
bay = BayesianRidge()
ridge = Ridge(alpha = 161)
extra = ExtraTreesRegressor(bootstrap = True, max_features = 'sqrt')
xgb = XGBRegressor(verbosity = 0)


In [94]:
# assign weights based on their gridsearch score

w1 = 0.25
w2 = 0.2
w3 = 0.2
w4 = 0.15
w5 = 0.15
w6 = 0.05

In [95]:
weight_avg = AverageWeight(mod = [ada,rf,bay,ridge,extra,xgb],weight=[w1,w2,w3,w4,w5,w6])

In [96]:
rmse_cv(weight_avg,X_scaled,y_log),  rmse_cv(weight_avg,X_scaled,y_log).mean()

(array([0.70856643, 0.67791396, 0.90970986, 0.96983565, 0.62672212]),
 0.7912477218510354)

# Stacking

In [76]:
class stacking(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self,mod,meta_model):
        self.mod = mod
        self.meta_model = meta_model
        self.kf = KFold(n_splits=5, random_state=42, shuffle=True)
        
    def fit(self,X,y):
        self.saved_model = [list() for i in self.mod]
        oof_train = np.zeros((X.shape[0], len(self.mod)))
        
        for i,model in enumerate(self.mod):
            for train_index, val_index in self.kf.split(X,y):
                renew_model = clone(model)
                renew_model.fit(X[train_index], y[train_index])
                self.saved_model[i].append(renew_model)
                oof_train[val_index,i] = renew_model.predict(X[val_index])
        
        self.meta_model.fit(oof_train,y)
        return self
    
    def predict(self,X):
        whole_test = np.column_stack([np.column_stack(model.predict(X) for model in single_model).mean(axis=1) 
                                      for single_model in self.saved_model]) 
        return self.meta_model.predict(whole_test)
    
    def get_oof(self,X,y,test_X):
        oof = np.zeros((X.shape[0],len(self.mod)))
        test_single = np.zeros((test_X.shape[0],5))
        test_mean = np.zeros((test_X.shape[0],len(self.mod)))
        for i,model in enumerate(self.mod):
            for j, (train_index,val_index) in enumerate(self.kf.split(X,y)):
                clone_model = clone(model)
                clone_model.fit(X[train_index],y[train_index])
                oof[val_index,i] = clone_model.predict(X[val_index])
                test_single[:,j] = clone_model.predict(test_X)
            test_mean[:,i] = test_single.mean(axis=1)
        return oof, test_mean

In [97]:
# must do imputer first, otherwise stacking won't work, and i don't know why.
a = Imputer().fit_transform(X_scaled)
b = Imputer().fit_transform(y_log.values.reshape(-1,1)).ravel()

In [98]:
stack_model = stacking(mod = [ada,rf,bay,ridge,extra,xgb], meta_model = ada)

In [99]:
print(rmse_cv(stack_model,a,b))
print(rmse_cv(stack_model,a,b).mean())

[0.64991879 0.71144264 0.88790144 1.03221626 0.65852377]
0.7883349738308209


### Extract the features generated from stacking, and combine them with original features.

In [100]:
X_train_stack, X_test_stack = stack_model.get_oof(a,b,test_X_scaled)

In [101]:
X_train_stack.shape, a.shape

((340, 6), (340, 26))

In [102]:
X_train_add = np.hstack((a,X_train_stack))

In [103]:
X_test_add = np.hstack((test_X_scaled,X_test_stack))

In [104]:
X_train_add.shape, X_test_add.shape

((340, 32), (90, 32))

In [105]:
print(rmse_cv(stack_model,X_train_add,b))
print(rmse_cv(stack_model,X_train_add,b).mean())

[0.74804753 0.67840787 0.89149094 1.05411525 0.70481074]
0.8457131979137005


# Submission

In [106]:
# Final Model

stack_model = stacking(mod = [ada,rf,bay,ridge,extra,xgb], meta_model = ada)

In [107]:
stack_model.fit(a,b)

stacking(meta_model=AdaBoostRegressor(base_estimator=None, learning_rate=0.06,
                                      loss='square', n_estimators=50,
                                      random_state=None),
         mod=[AdaBoostRegressor(base_estimator=None, learning_rate=0.06,
                                loss='square', n_estimators=50,
                                random_state=None),
              RandomForestRegressor(bootstrap=True, criterion='mse',
                                    max_depth=None, max_features='log2',
                                    max_leaf_nodes=None,
                                    min_i...
              XGBRegressor(base_score=0.5, booster='gbtree',
                           colsample_bylevel=1, colsample_bynode=1,
                           colsample_bytree=1, gamma=0, importance_type='gain',
                           learning_rate=0.1, max_delta_step=0, max_depth=3,
                           min_child_weight=1, missing=None, n_estimators=

In [108]:
pred = np.exp(stack_model.predict(test_X_scaled) + 1)

In [109]:
result=pd.DataFrame({'id':range(len(df_test2)), 'playtime_forever':pred})


In [111]:
result.to_csv("./Submission/submission_1120_Mode(2).csv",index=False)